# XLNet를 이용한 감정 분석
사전학습모델 : [xlnet-base-cased](https://huggingface.co/xlnet-base-cased) <br>
데이터 : [GLUE_SST-2 (Stanford Sentiment Treebank v2)](https://huggingface.co/datasets/glue/viewer/sst2/train)

# 사전 준비

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

**GLUE의 SST-2 데이터 불러오기**

In [2]:
from datasets import load_dataset

datasets = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [4]:
# label 0: negative(부정) / 1: positive(긍정) / -1: test data (비공개)
print(datasets["train"][0])
print(datasets["validation"][0])
print(datasets["test"][0])

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}
{'sentence': "it 's a charming and often affecting journey . ", 'label': 1, 'idx': 0}
{'sentence': 'uneasy mishmash of styles and genres .', 'label': -1, 'idx': 0}


**XLNet 모델과 토크나이저 불러오기**

In [5]:
from transformers import XLNetTokenizer, XLNetModel

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased')

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model.config

XLNetConfig {
  "_name_or_path": "xlnet-base-cased",
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "transformers_version": "4.21.3",
  "untie_r": true,
  "use_mems_eval": true,
  "use_mems_train": false,
  "vocab_size": 32000
}

# 데이터 구축

**데이터 준비** <br>
train:validation:test = 7 : 1 : 2

In [7]:
from tqdm.auto import tqdm as tqdm_auto

In [8]:
ids = datasets['validation'].num_rows

In [9]:
train_sentence = [datasets['train']['sentence'][idx] for idx in tqdm_auto(range(0, ids*7))]
train_label = [datasets['train']['label'][idx] for idx in tqdm_auto(range(0, ids*7))]

  0%|          | 0/6104 [00:00<?, ?it/s]

  0%|          | 0/6104 [00:00<?, ?it/s]

In [12]:
val_sentence = [datasets['validation']['sentence'][idx] for idx in tqdm_auto(range(0, ids))]
val_label = [datasets['validation']['label'][idx] for idx in tqdm_auto(range(0, ids))]

  0%|          | 0/872 [00:00<?, ?it/s]

  0%|          | 0/872 [00:00<?, ?it/s]

In [13]:
# SST-2의 test data는 비공개로 train data의 일부로 test data를 만든다
test_sentence = [datasets['train']['sentence'][idx] for idx in tqdm_auto(range(ids*7, ids*9))]
test_label = [datasets['train']['label'][idx] for idx in tqdm_auto(range(ids*7, ids*9))]

  0%|          | 0/1744 [00:00<?, ?it/s]

  0%|          | 0/1744 [00:00<?, ?it/s]

In [18]:
# 마지막 train data와 test data의 마지막과 처음이 중복인지 확인
print("last train data:", train_sentence[-1])
print("last test data:", test_sentence[0])

last train data: a moral 
last test data: that gives movies about ordinary folk a bad name 


**토크나이징**

In [19]:
# 패딩 채우기
train_input = tokenizer(train_sentence, padding=True, return_tensors="pt")
val_input = tokenizer(val_sentence, padding=True, return_tensors="pt")
test_input = tokenizer(test_sentence, padding=True, return_tensors="pt")